In [29]:
from datasets import load_dataset
import torch


dataset = load_dataset("dair-ai/emotion") # Esta línea no funciona en Cousera, es para que hagan su descarga en Colab o en su entorno personal

### Vamos a trabajar con el dataset dair-ai/emotion de Hugging Face. 

- Usaremos el modelo distilbert-base-uncased para hacer fine-tuning en el dataset y, con esto, poder realizar la clasificación de sentimientos. La estructura del dataset y cómo trabajar con los datos tendrán que investigarla ustedes, pero estoy seguro de que serán capaces =D.

## Para este proyecto tendrán que investigar y aplicar el uso de los siguientes módulos de HuggingFace
- AutoTokenizer o DistilBertTokenizer
- AutoConfig o DistilBertConfig
- AutoModelForSequenceClassification o DistilBertForSequenceClassification
- DataCollatorWithPadding o hacer su propia función collatefn

## No es obligatorio usar AutoTokenizer, AutoConfig, etc. Pueden hacerlo de otras formas, como entrenando su propio Tokenizer en su propio modelo y/o vocabulario y de ahí entrenar el Transformer para hacer la clasificación. **Esto es mucho más desafiante, no lo recomiendo si disponen de poco tiempo**. Así que recomiendo que usen las estructuras que ya existen de HuggingFace, para cumplir con la complejidad esperada para el curso.

## Les recomiendo fuertemente entrenar en un Google Colab, o en su propia máquina si ya tienen el entorno configurado.

La calificación será en torno a métricas conocidas de sklearn:
```python
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

accuracy = accuracy_score(real_labels, predictions)
precision, recall, f1, _ = precision_recall_fscore_support(real_labels, predictions, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
```
score = $(Accuracy + Precision + Recall + F1 Score)/4
$

Son 5 posibles salidas, un modelo que predice aleatoriamente tendría un 20% de precisión, así que empezaremos con un 50% y subiremos gradualmente hasta el 90%.

- 50% = 10/100 pts
- 55% = 20/100 pts
- 60% = 30/100 pts
- 65% = 40/100 pts
- 70% = 50/100 pts
- 75% = 60/100 pts
- 80% = 70/100 pts
- 85% = 90/100 pts
- 90% = 100/100 pts # Mi algoritmo con pocas modificaciones alcanza el 93%, así que ustedes ¡sí o sí pueden!


# **Importante**
Nuevamente, como les he mencionado, es importante que entrenen en un Colab aparte. Para esto, una vez que terminen el entrenamiento, tendrán que guardar los pesos del modelo y cargarlos en el evaluador. Si no saben cómo hacerlo, les recomiendo fuertemente ver el video tutorial sobre esto.

Además tambien es necesario que la función collate_fn, sea la que implementarón ustedes o DataCollatorWithPadding sea ejecutada en el archivo con nombre **data_collator**, la instancia del modelo **model**, la función de tokenize **tokenize_function** ( Esto es, la función que se utiliza para definir y llamar el tokenizer para cada 'batch'

## Respuesta Tarea

Separaremos el dataset entre entrenamiento, test y validación

In [1]:
from datasets import load_dataset
import torch


dataset = load_dataset("dair-ai/emotion")
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [2]:
train_dataset = dataset['train']
eval_dataset = dataset['validation']
test_dataset = dataset['test']

In [3]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 16000
})

Definimos el nombre de nuestro modelo para luego utilizar el tokenizador y el modelo. También, definimos el número de clases.

In [3]:
model_name = 'distilbert-base-uncased'
num_classes = len(set(train_dataset['label']))
num_classes

6

Cargamos el tokenizador y pasamos los datos a tensores

In [4]:
from transformers import AutoTokenizer, DataCollatorWithPadding

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Creamos una función para tokenizar el texto y luego generar un tensor por cada dato

In [6]:
def tokenize_function(dataset):
    return tokenizer(dataset['text'], truncation=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Definimos el device a utilizar. Como estoy en un Mac, la GPU es `mps`

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'mps')

Ahora, definimos el modelo

In [8]:
from transformers import AutoModelForSequenceClassification

In [9]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

### Fine-tuning del modelo

Entrenaremos el modelo en base a nuestro dataset de entrenamiento `train_dataset` y validaremos con `val_dataset`

In [19]:
from transformers import TrainingArguments, Trainer

Definimos los parámetros para el entrenamiento

In [21]:
model_output_path = 'finetuned_model_ldavico'
learning_rate = 1e-4
per_device_train_batch_size = 32
per_device_eval_batch_size = 32
num_train_epochs = 5
weight_decay = 0.01

Tokenizamos nuestras frases

In [22]:
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)

In [23]:
training_args = TrainingArguments(
   output_dir=model_output_path,
   learning_rate=learning_rate,
   per_device_train_batch_size=per_device_train_batch_size,
   per_device_eval_batch_size=per_device_eval_batch_size,
   num_train_epochs=num_train_epochs,
   weight_decay=weight_decay,
   save_strategy="epoch",
)
 
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_eval,
   tokenizer=tokenizer,
   data_collator=data_collator,
)

In [24]:
trainer.train()

  0%|          | 0/2500 [00:00<?, ?it/s]

{'loss': 0.4023, 'learning_rate': 8e-05, 'epoch': 1.0}
{'loss': 0.1469, 'learning_rate': 6e-05, 'epoch': 2.0}
{'loss': 0.1033, 'learning_rate': 4e-05, 'epoch': 3.0}
{'loss': 0.0704, 'learning_rate': 2e-05, 'epoch': 4.0}
{'loss': 0.0367, 'learning_rate': 0.0, 'epoch': 5.0}
{'train_runtime': 597.9344, 'train_samples_per_second': 133.794, 'train_steps_per_second': 4.181, 'train_loss': 0.15190772399902344, 'epoch': 5.0}


TrainOutput(global_step=2500, training_loss=0.15190772399902344, metrics={'train_runtime': 597.9344, 'train_samples_per_second': 133.794, 'train_steps_per_second': 4.181, 'train_loss': 0.15190772399902344, 'epoch': 5.0})

### Exportamos los pesos y configuración

In [30]:
exported_model_name_path = 'finetuned_model'

model.save_pretrained(exported_model_name_path)

### Evaluación

In [39]:
import torch
import datasets
from transformers import DistilBertForSequenceClassification
from pandas import read_csv
from torch.utils.data import DataLoader
from transformers import DistilBertTokenizer, DataCollatorWithPadding

# Cargar tu modelo
model = AutoModelForSequenceClassification.from_pretrained(exported_model_name_path, num_labels=num_classes)
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [40]:
# Este dataset ya está cargado arriba
#eval_dataset = read_csv('test.csv')

Acá se utilizó el `test_dataset` procesado de la misma forma que el `eval_dataset`. Además, había corregí la columna llamada `labels` a `label`

In [41]:
# Si hiciste alguno cambio distinto de lo propuesto abajo, cambiar como se procesa el set de testeo para obtener el mismo tipo de datos

test_dataset = test_dataset.map(tokenize_function, batched=True)
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [42]:
test_dataloader = DataLoader(test_dataset, batch_size=8, collate_fn=data_collator)
model.eval()
predictions = []
real_labels = []

for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    # Acá si el formato que dejaste los batches es distinto, adecuar funcion
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1)
    # NO EDITAR    
    predictions.extend(predicted_labels.cpu().numpy())
    real_labels.extend(batch['labels'].cpu().numpy())


In [44]:

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

accuracy = accuracy_score(real_labels, predictions)
precision, recall, f1, _ = precision_recall_fscore_support(real_labels, predictions, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print('score: ', (accuracy + precision + recall + f1) / 4)

Accuracy: 0.928
Precision: 0.9269373615544183
Recall: 0.928
F1 Score: 0.9272691021082526
score:  0.9275516159156678
